In [ ]:
# =========================================================
# 📦 IMPORTACIONES
# =========================================================
import os
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras import layers, optimizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# =========================================================
# ⚙️ CONFIGURACIÓN DE GPU
# =========================================================
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        dev_names = [tf.config.experimental.get_device_details(g).get("device_name", str(g)) for g in gpus]
        print(f"✅ GPUs detectadas: {len(gpus)} -> {dev_names}")
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(f"   Logical GPUs: {len(logical_gpus)}")
    except RuntimeError as e:
        print("⚠️ Error al configurar GPUs:", e)
else:
    print("⚠️ No se detectó GPU, se usará CPU")

# --- Opcional: activar mixed precision ---
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')
# (si usas esto, recuerda que la última capa Dense debe ser dtype='float32')

In [ ]:
# =========================================================
# 📂 CARGAR RUTAS
# =========================================================

# Rutas actualizadas para la nueva estructura
BASE_DIR = "images"
META_DIR = "meta/meta"

# Verificar que las rutas existen
if os.path.exists(BASE_DIR):
    print(f"Se encontró la ruta: {BASE_DIR} exitosamente!")
if os.path.exists(META_DIR):
    print(f"Se encontró la ruta: {META_DIR} exitosamente!")

In [ ]:
# =========================================================
# 📄 LECTURA DE SPLITS (train/test)
# =========================================================
def load_split(filename):
    path = os.path.join(META_DIR, filename).replace("\\", "/")
    try:
        with open(path, "r") as f:
            lines = f.read().splitlines()
        return lines
    except FileNotFoundError:
        print(f"❌ Error: No se encontró el archivo {path}")
        return []

train_files = load_split("train.txt")
test_files = load_split("test.txt")

print(f"📊 Train muestras: {len(train_files)}")
print(f"📊 Test muestras: {len(test_files)}")

if len(train_files) == 0 or len(test_files) == 0:
    print("❌ Error: No se pudieron cargar los archivos de splits")

In [ ]:
# =========================================================
# 📊 CREAR DATAFRAMES PARA TRAIN Y TEST
# =========================================================
def create_dataframe(file_list, base_dir):
    """Crear DataFrame con rutas de imágenes y etiquetas"""
    data = []
    for file_path in file_list:
        # Extraer clase del nombre del archivo (formato: clase/imagen.jpg)
        parts = file_path.split('/')
        if len(parts) >= 2:
            class_name = parts[0]
            full_path = os.path.join(base_dir, file_path).replace("\\", "/") + ".jpg"
            data.append({
                'filename': full_path,
                'class': class_name
            })
    
    return pd.DataFrame(data)

# Crear DataFrames
train_df = create_dataframe(train_files, BASE_DIR)
test_df = create_dataframe(test_files, BASE_DIR)

print(f"📊 Train DataFrame shape: {train_df.shape}")
print(f"📊 Test DataFrame shape: {test_df.shape}")
print(f"📊 Número de clases: {train_df['class'].nunique()}")
print(f"📊 Clases encontradas: {sorted(train_df['class'].unique())}")

# Verificar algunas imágenes
print("\n🔍 Verificando existencia de algunas imágenes:")
sample_files = train_df['filename'].head().tolist()
for file_path in sample_files:
    exists = os.path.exists(file_path)
    print(f"{'✅' if exists else '❌'} {file_path}")

In [ ]:
# =========================================================
# 🏗️ CONFIGURACIÓN DE GENERADORES DE DATOS
# =========================================================
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# Dividir train_df en train y validation
train_split, val_split = train_test_split(
    train_df, 
    test_size=0.2, 
    stratify=train_df['class'],  # Mantener proporciones de clases
    random_state=42
)

print(f"📊 Train split: {len(train_split)} muestras")
print(f"📊 Validation split: {len(val_split)} muestras")

# Data Augmentation reducido para entrenamiento
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    zoom_range=0.1,
    fill_mode='nearest'
)

# Solo preprocessing para validación
val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

# Solo preprocessing para test
test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

# Crear generadores usando los splits separados
train_generator = train_datagen.flow_from_dataframe(
    train_split,
    x_col='filename',
    y_col='class',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_dataframe(
    val_split,
    x_col='filename',
    y_col='class',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

test_generator = test_datagen.flow_from_dataframe(
    test_df,
    x_col='filename',
    y_col='class',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

train_steps = len(train_generator)
val_steps = len(val_generator)    
test_steps = len(test_generator)  

print("✅ Generadores corregidos:")
print(f"📊 Train steps: {train_steps}")
print(f"📊 Validation steps: {val_steps}")
print(f"📊 Test steps: {test_steps}")
print(f"📊 Total train images: {train_generator.samples}")
print(f"📊 Total validation images: {val_generator.samples}")
print(f"📊 Total test images: {test_generator.samples}")
print(f"📊 Número de clases: {len(train_generator.class_indices)}")

# Verificar un batch
try:
    sample_batch = next(train_generator)
    print(f"✅ Batch de prueba: {sample_batch[0].shape}, {sample_batch[1].shape}")
except Exception as e:
    print(f"❌ Error al obtener batch de prueba: {e}")

In [ ]:
# =========================================================
# 🏗️ CONSTRUCCIÓN DEL MODELO ResNet50
# =========================================================
NUM_CLASSES = len(train_generator.class_indices)

tf.keras.backend.clear_session()

# Cargar ResNet50 pre-entrenado sin la capa top
base_model = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(*IMG_SIZE, 3)
)

# Agregar capas personalizadas
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.BatchNormalization()(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.3)(x)
predictions = layers.Dense(NUM_CLASSES, activation='softmax', name='predictions', dtype='float32')(x)

# Crear el modelo completo
model = Model(inputs=base_model.input, outputs=predictions)

print(f"✅ Modelo corregido con {NUM_CLASSES} clases")
print(f"📊 Parámetros totales: {model.count_params():,}")

# Mostrar resumen de capas trainables/no trainables
trainable_params = sum([tf.size(weight).numpy() for weight in model.trainable_weights])
non_trainable_params = sum([tf.size(weight).numpy() for weight in model.non_trainable_weights])
print(f"📊 Parámetros entrenables: {trainable_params:,}")
print(f"📊 Parámetros no entrenables: {non_trainable_params:,}")

In [ ]:
# =========================================================
# 🎯 ENTRENAMIENTO POR FASES
# =========================================================

# ---- Fase 1: Entrenar solo el clasificador ----
print("\n" + "="*50)
print("🔒 FASE 1: Entrenando solo el clasificador")
print("="*50)

# Congelar ResNet50
for layer in base_model.layers:
    layer.trainable = False

# Compilar para Fase 1
model.compile(
    optimizer=Adam(learning_rate=1e-3),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

# Callbacks mejorados
early_stop_fase1 = EarlyStopping(
    monitor="val_accuracy",
    patience=5,
    restore_best_weights=True,
    verbose=1,
    mode='max'
)

checkpoint_fase1 = ModelCheckpoint(
    "best_model_fase1.h5",
    monitor="val_accuracy",
    save_best_only=True,
    verbose=1,
    mode='max'
)

# Callback para reducir learning rate
reduce_lr_fase1 = ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.2,
    patience=3,
    min_lr=1e-7,
    verbose=1,
    mode='max'
)

# Entrenar Fase 1
print("🚀 Entrenando Fase 1 (solo capas densas)")
history_fase1 = model.fit(
    train_generator,
    steps_per_epoch=train_steps,
    validation_data=val_generator,
    validation_steps=val_steps,
    epochs=15,
    callbacks=[early_stop_fase1, checkpoint_fase1, reduce_lr_fase1],
    verbose=1
)

# ---- Fase 2: Fine-tuning ----
print("\n" + "="*50)
print("🔓 FASE 2: Fine-tuning ")
print("="*50)

# Desbloquear gradualmente las capas de ResNet50
trainable = False
for layer in base_model.layers:
    # Desbloquear a partir de 'conv5_block1_1_conv' (últimas capas del ResNet)
    if layer.name == 'conv5_block1_1_conv':
        trainable = True
    layer.trainable = trainable

print(f"Capas desbloqueadas: {sum(1 for layer in base_model.layers if layer.trainable)}")

# Compilar para Fase 2 con learning rate más bajo
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

# Callbacks para Fase 2
early_stop_fase2 = EarlyStopping(
    monitor="val_accuracy",
    patience=8,
    restore_best_weights=True,
    verbose=1,
    mode='max'
)

checkpoint_fase2 = ModelCheckpoint(
    "best_model_fase2.h5",
    monitor="val_accuracy",
    save_best_only=True,
    verbose=1,
    mode='max'
)

# Callback para reducir learning rate en fase 2
reduce_lr_fase2 = ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.5,
    patience=4,
    min_lr=1e-8,
    verbose=1,
    mode='max'
)

# Entrenar Fase 2
print("🚀 Entrenando Fase 2 (fine-tuning) - CORREGIDO...")
history_fase2 = model.fit(
    train_generator,
    steps_per_epoch=train_steps,
    validation_data=val_generator,
    validation_steps=val_steps,
    epochs=20,
    callbacks=[early_stop_fase2, checkpoint_fase2, reduce_lr_fase2],
    verbose=1
)

print("✅ Entrenamiento completado!")

# =========================================================
# 📊 EVALUACIÓN EN TEST SET
# =========================================================
print("\n" + "="*50)
print("📊 EVALUACIÓN FINAL")
print("="*50)

# Cargar el mejor modelo
model.load_weights("best_model_fase2.h5")

# Evaluar en test set
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_steps, verbose=1)

print(f"\n🎯 RESULTADOS FINALES:")
print(f"📊 Test Loss: {test_loss:.4f}")
print(f"📊 Test Accuracy: {test_accuracy:.4f}")

# Predicciones detalladas
test_generator.reset()
predictions = model.predict(test_generator, steps=test_steps, verbose=1)
predicted_classes = np.argmax(predictions, axis=1)

# Obtener clases reales
test_generator.reset()
true_classes = test_generator.classes

# Classification report
class_names = list(test_generator.class_indices.keys())

print(f"\n📊 Classification Report:")
print(classification_report(true_classes, predicted_classes, target_names=class_names))

In [ ]:
# =========================================================
# 💾 GUARDADO DEL MODELO FINAL
# =========================================================
model_path = "model_resnet50_final.h5"
model.save(model_path)
print(f"✅ Modelo final guardado en: {model_path}")

# Guardar también los pesos por separado
weights_path = "model_resnet50_weights.h5"
model.save_weights(weights_path)
print(f"✅ Pesos guardados en: {weights_path}")